In [1]:
import pandas as pd
file = pd.read_excel('russian_speech.xlsx')
y = [sentence for sentence in file['Русская речь']]

In [2]:
import os
import librosa

dir_name = "abnormal_voice/"
files_in_dir = [f for f in os.listdir(dir_name)]

X = []
i = 1
for e in files_in_dir:
    X.append(librosa.load(dir_name + e, sr=16000)[0])
    if i % 100 == 0:
        print(i)
    i += 1

100
200
300
400
500
600
700
800
900
1000


In [3]:
X_train = X[:700]
y_train = y[:700]
X_test = X[700:]
y_test = y[700:]

In [4]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\\xad\\n\–]'

def remove_special_characters(sentence):
    sentence = re.sub(chars_to_ignore_regex, '', sentence).lower() + " "
    sentence = sentence.replace('4', 'четыре').replace('р220', 'р двести двадцать').replace('6', 'шесть')
    return sentence

y = list(map(remove_special_characters, y))
y_train = list(map(remove_special_characters, y_train))
y_test = list(map(remove_special_characters, y_test))

In [7]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

In [17]:
model = AutoModelForCTC.from_pretrained("UrukHan/wav2vec2-russian")
processor = Wav2Vec2Processor.from_pretrained("UrukHan/wav2vec2-russian")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
from datasets import Dataset
def p_d(audioX, textY):
    X_list = []
    Len_list = []
    Y_list = []
    for i in range(len(audioX)):
        X_list.append(processor(audioX[i], sampling_rate=16000).input_values[0])
        Len_list.append(len(X_list[-1]))
        with processor.as_target_processor():
            Y_list.append(processor(textY[i]).input_ids)
        
    dataset = Dataset.from_dict({'input_values': X_list, 'input_length': Len_list, 'labels': Y_list})
    return dataset

In [10]:
train_dataset = p_d(X_train, y_train)
test_dataset = p_d(X_test, y_test)

C:\source\tenserflow\venv\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [12]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """
        
    def __init__(self, processor, padding):
        self.processor = processor
        self.padding = padding

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [13]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [14]:
from datasets import load_metric
wer_metric = load_metric("wer")

C:\Users\Roman\AppData\Local\Temp\ipykernel_5672\2912728219.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [15]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [19]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="UrukHan/wav2vec2-russian",
  group_by_length=True,
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=False,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-7,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=False
)

In [20]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

In [21]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
C:\source\tenserflow\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 700
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 440
  Number of trainable parameters = 315475620
C:\source\tenserflow\venv\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated 

Step,Training Loss,Validation Loss


KeyboardInterrupt: 